In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
from io import StringIO
from pandas.api.types import is_numeric_dtype
import requests as req
from bs4 import BeautifulSoup
import json
import unidecode

import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from warnings import warn

In [3]:
op = webdriver.ChromeOptions()

op.add_argument('--headless')

driver = webdriver.Chrome('/Applications/chromedriver', options=op)
driver2 = webdriver.Chrome('/Applications/chromedriver', options=op)

# link = 'https://web.archive.org/web/20210126114928/https://www.ons.gov.uk/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25'

# driver.get(link)

# content = ""

# content = content + driver.find_element_by_xpath("//*[@id='main']/div[2]/div[1]/article/div[2]/section/h2").text + "\n"

# ps = driver.find_elements_by_xpath("//*[@id='main']/div[2]/div[1]/article/div[2]/section//p")

# for i, j in enumerate(ps):
#     if (i != 0) & (j.text != '') & (j.text not in ['', 'Back to the top', 'Analysis | Data']):
#         content = content + j.text + "\n"
# content

In [4]:
# ONS LATEST INSIGHTS APRIL Wayback machine archive urls
# url = 'http://web.archive.org/cdx/search/cdx?url=https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights&collapse=digest&from=20210709&to=20210801&output=json'
# url = 'http://web.archive.org/cdx/search/cdx?url=ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26&collapse=digest&from=20200801&to=20210101&output=json'
url = 'http://web.archive.org/cdx/search/cdx?url=ons.gov.uk/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25&collapse=digest&from=20200101&to=20220601&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls) #parses the JSON from urls.

## Extracts timestamp and original columns from urls and compiles a url list.
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
## Compiles final url pattern.
final_urls = []
for url in url_list:
    final_urls.append('https://web.archive.org/web/'+url)

In [79]:
d = {'page': [], 'roundup': [], 'bulletin_url': [], 'bulletin_mps': []}
df = pd.DataFrame(data=d)
df

,page,roundup,bulletin_url,bulletin_mps


In [85]:
def getbulurl(link, n):
    driver2.get(link)
    
    ps = driver2.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//p")
    
    date = [i.text for i in ps if len(i.text) < 18][0]
    reldate = "Released: " + date.rstrip()
    
    print(reldate)
        
    atags = [i for i in driver2.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//a[1]")]
    atags.reverse()
    words = ['Back to the top', 'Data']
    for word in words:
        atags = [i for i in atags if (word not in i.text)]
        
    url1 = [i.get_attribute("href") for i in atags][0]
    
    if url1.split("/")[-1] == 'latest':
        print(url1)
        driver2.get(url1)
        urlPrev = driver2.find_element_by_link_text("View previous releases").get_attribute("href")
        
        url = 'null'
        
        for i in range(1, 20):
            driver2.get(urlPrev + "?page=" + str(i) )
            lis = driver2.find_elements_by_xpath('//*[@id="main"]/div[2]/div/div[1]/ul/li')
            listoas = [i.find_element_by_tag_name("a") for i in lis if i.find_element_by_tag_name("p").text == reldate]
            if len(listoas) > 0:
                url = listoas[0].get_attribute("href")
                break
    
        if url == 'null':
                
            urlPrev = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'

            for i in range(1, 20):
                driver2.get(urlPrev + "?page=" + str(i) )
                lis = driver2.find_elements_by_xpath('//*[@id="main"]/div[2]/div/div[1]/ul/li')
                listoas = [i.find_element_by_tag_name("a") for i in lis if i.find_element_by_tag_name("p").text == reldate]
                if len(listoas) > 0:
                    url = listoas[0].get_attribute("href")
                    break
 
    else:
        url = url1
    
    return url

getbulurl('https://web.archive.org/web/20210126114928/https://www.ons.gov.uk/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25', 3)

Released: 26 January 2021
https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/earningsandemploymentfrompayasyouearnrealtimeinformationuk/latest


'https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/earningsandemploymentfrompayasyouearnrealtimeinformationuk/january2021'

In [86]:
def getru(link, df):

    driver.get(link)    
    
    for n in range(2, 8):
        print('n', n)
        
        if driver.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section/h2"):
        
            content = ""

            content = content + driver.find_element_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section/h2").text + "\n"

            ps = driver.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//p")

            for i, j in enumerate(ps):
                if (i != 0) & (j.text != '') & (j.text not in ['', 'Back to the top', 'Analysis | Data']):
                    content = content + j.text + "\n"

            if content not in [i for i in df['roundup']]:
                df = df.append({'page': link, 'roundup': content, 'bulletin_url': getbulurl(link, n), 'bulletin_mps': 'placeholder'}, ignore_index=True)
    
    return df

In [202]:
len(final_urls)

26

In [203]:
final_urls[13]

'https://web.archive.org/web/20220222092131/https://www.ons.gov.uk/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25'

In [204]:
driver.get(final_urls[13])

In [205]:
driver.find_element_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[2]/section/h2").text

'Borrowing in the financial year-to-date halved compared with a year ago'

In [107]:
df

,page,roundup,bulletin_url,bulletin_mps
0,https://web.archive.org/web/20210126034402/htt...,"Retail sales rose in December, but saw a recor...",https://www.ons.gov.uk/businessindustryandtrad...,placeholder
1,https://web.archive.org/web/20210126034402/htt...,Highest December borrowing since records began...,https://www.ons.gov.uk/economy/governmentpubli...,placeholder
2,https://web.archive.org/web/20210126034402/htt...,Coronavirus hits young and poorly-paid furloug...,https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
3,https://web.archive.org/web/20210126034402/htt...,UK debit and credit card purchases 35% below t...,https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
4,https://web.archive.org/web/20210126034402/htt...,Average house price growth increased further i...,https://www.ons.gov.uk/economy/inflationandpri...,placeholder
...,...,...,...,...
152,https://web.archive.org/web/20220413061940/htt...,Prices at the pump drive up inflation\nThe Con...,https://www.ons.gov.uk/releases/consumerpricei...,placeholder
153,https://web.archive.org/web/20220417162327/htt...,UK average house prices increased by 10.9% ove...,https://www.ons.gov.uk/releases/ukhousepricein...,placeholder
154,https://web.archive.org/web/20220417162327/htt...,Prices at the pump drive up inflation\nThe Con...,https://www.ons.gov.uk/releases/consumerpricei...,placeholder
155,https://web.archive.org/web/20220427165340/htt...,ONS improving inflation figures\nFollowing the...,https://www.ons.gov.uk/releases/transformation...,placeholder


In [99]:
final_urls = [x for x in final_urls if x not in [i for i in df['page']][:-1]]

In [100]:
for url in final_urls:
    df = getru(url, df)

n 2
n 3
n 4
n 5
n 6
n 7
n 2
n 3
n 4
n 5
n 6
n 7
n 2
n 3
n 4
n 5
n 6
n 7
n 2
n 3
n 4
n 5
n 6
n 7
n 2
n 3
n 4
n 5
n 6
n 7
n 2
n 3
n 4
n 5
n 6
n 7
n 2
Released: 
n 3
Released: 🠕 Back to the top
https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/uklabourmarket/latest
n 4
Released: 🠕 Back to the top
https://www.ons.gov.uk/economy/economicoutputandproductivity/productivitymeasures/articles/gdpandthelabourmarket/latest
n 5
Released: 
n 6
Released: 
https://www.ons.gov.uk/economy/grossdomesticproductgdp/bulletins/gdpmonthlyestimateuk/latest
n 7
Released: 
https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/bulletins/uktrade/latest
n 2
Released: 
https://www.ons.gov.uk/economy/inflationandpriceindices/bulletins/housepriceindex/latest
n 3
n 4
n 5
n 6
n 7
n 2
Released: 🠕 Back to the top
https://www.ons.gov.uk/economy/governmentpublicsectorandtaxes/publicsectorfinance/bulletins/publicsectorfinances/latest
n 3
Released: 
https://www.ons.

In [108]:
dfurls = df[df['bulletin_url'] != 'null']

In [111]:
def getmps(bulletinurl):
    
    # Open page for roundup
    req = rq.get(bulletinurl).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')
        
    if soup.find('h2'):
        if soup.find('h2').text == 'Publications':
            url = 'https://www.ons.gov.uk' + soup.find('h3', class_="tiles__title-h3").find("a")['href']
            req = rq.get(url).text
            soup = bs(req,'html.parser')
        

    sects = [sect for sect in soup.find_all('section') if sect.find_all('h2') != []]    
    sect = [sect for sect in sects if 'Main points' in sect.find('h2').text]
    
    if len(sect)>0:
                
        sect = sect[0]

        lis = sect.find('ul')
        if lis != None:
            sibling = lis.next_sibling

            while sibling:
                next_sibling = sibling.next_sibling
                sibling.extract()
                sibling = next_sibling
            
        ps = []
        for i in sect:
            if i.name == 'p':
                ps.append(i.text)

            elif i.name == 'ul':
                for j in i.find_all('li'):
                    ps.append(j.text)            
            
    
    else:
        return "NO MAIN POINTS"
        
    mps = [i for i in ps if ('Source: Office for National Statistics' not in i) and ('Find out more' not in i) and ('make the website work as well as possible' not in i) and ('Embed this interactive' not in i) and ('xlsx' not in i) and ('cookies' not in i)]
    mainpoints = ""
    for i in mps: 
        mainpoints = mainpoints + i + '\n' 
    
    return mainpoints

print(getmps(dfurls['bulletin_url'][150]))

Exports of goods to EU countries, excluding precious metals, increased notably in February 2022, following on from the 20.5% decrease in January that was attributed to the operational change implemented by HM Revenue and Customs (HMRC) in January 2022; we therefore recommend users do not rely on these month-on-month movements when interpreting the latest monthly data for exports to the EU.
Total exports of goods, excluding precious metals, increased by £2.1 billion (7.8%) in February 2022, driven by a £3.0 billion (25.4%) increase in exports to EU countries while exports to non-EU countries decreased by £0.9 billion (6.4%).
While exports of goods to EU countries, excluding precious metals, have increased by 25.4% in February 2022 compared with January 2022, exports to the EU have decreased by 0.3% when comparing February 2022 with December 2021.
Total imports of goods, excluding precious metals, decreased by £1.0 billion (2.2%) in February 2022, because of a £0.7 billion (3.0%) fall in

In [112]:
for i in dfurls.index:
    if dfurls.loc[i]['bulletin_mps'] == 'placeholder':
        print(dfurls.loc[i]['bulletin_url'])
        dfurls.loc[i]['bulletin_mps'] = getmps(dfurls.loc[i]['bulletin_url'])

https://www.ons.gov.uk/chartimage?uri=/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25/50b28d5d
https://www.ons.gov.uk/releases/employeeearningsintheuk2021
https://www.ons.gov.uk/economy/governmentpublicsectorandtaxes/publicsectorfinance/bulletins/publicsectorfinances/september2021
https://www.ons.gov.uk/economy/inflationandpriceindices/articles/pricemovementsinconstructionmaterialsandplanthireuk/2019to2021
https://www.ons.gov.uk/releases/internationaltradebysubnationalareasoftheuk2019
https://www.ons.gov.uk/releases/economicactivityandsocialchangeintheukrealtimeindicators4november2021
https://www.ons.gov.uk/businessindustryandtrade/business/businessservices/bulletins/businessinsightsandimpactontheukeconomy/4november2021
https://www.ons.gov.uk/releases/internationaltradebysubnationalareasoftheuk2019
https://www.ons.gov.uk/releases/economicactivityandsocialchangeintheukrealtimeindicators11november2021
https://www.ons.gov.uk/releases/economicactivityandso

In [120]:
[i for i in dfurls['bulletin_mps'] if len(i) <150]

['NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS',
 'NO MAIN POINTS']

In [106]:
df[df['bulletin_url']!= 'null']

In [122]:
df_mps = dfurls[dfurls['bulletin_mps'] != 'NO MAIN POINTS']

In [125]:
df_mps['rougef1'] = 'ph'

<ipython-input-125-af59cf3707bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mps['rougef1'] = 'ph'


In [127]:
from rouge import Rouge 
rouge = Rouge()

In [129]:
for i in df_mps.index:
    df_mps['rougef1'][i] = rouge.get_scores(df_mps.loc[i]['roundup'], df_mps.loc[i]['bulletin_mps'])[0]['rouge-2']['f']

/Users/theojolliffe/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [179]:
df_mps['len_ratio'] = 'ph'

In [190]:
df_mps

,page,roundup,bulletin_url,bulletin_mps,rougef1,len_ratio
59,https://web.archive.org/web/20210605155126/htt...,Number of UK seated diners up by 41 percentage...,https://www.ons.gov.uk/releases/traveltrends2020,Overseas residents made 11.1 million visits to...,0.0135135,0.594473
109,https://web.archive.org/web/20211117082737/htt...,Price of energy drives up inflation to 3.8%\nT...,https://www.ons.gov.uk/releases/internationalt...,"In 2019, the South East of England led the UK ...",0.0168776,2.34161
55,https://web.archive.org/web/20210512105154/htt...,Seated dining levels continue to rise as UK re...,https://www.ons.gov.uk/peoplepopulationandcomm...,This week's bulletin is reduced because of the...,0.0175439,0.737853
80,https://web.archive.org/web/20210811194228/htt...,More than half of businesses highly confident ...,https://www.ons.gov.uk/economy/economicoutputa...,"In the week to 25 July 2021, the seven-day ave...",0.0429338,4.23293
105,https://web.archive.org/web/20211101034726/htt...,Increased prices in wood and steel partly beca...,https://www.ons.gov.uk/economy/inflationandpri...,Producer Price Inflation (PPI) inputs into con...,0.0495356,0.333977
...,...,...,...,...,...,...
52,https://web.archive.org/web/20210512105154/htt...,The economy grew by 2.1% in March 2021\nReal g...,https://www.ons.gov.uk/economy/grossdomesticpr...,UK gross domestic product (GDP) is estimated t...,0.631579,0.630137
6,https://web.archive.org/web/20210126114928/htt...,There is a large increase in the unemployment ...,https://www.ons.gov.uk/employmentandlabourmark...,"Since February 2020, the number of payroll emp...",0.665605,1.60689
151,https://web.archive.org/web/20220412224338/htt...,Employment rate remains below pre-pandemic lev...,https://www.ons.gov.uk/releases/uklabourmarket...,The latest Labour Force Survey (LFS) estimates...,0.676519,1.4917
150,https://web.archive.org/web/20220411100743/htt...,Total imports and exports of goods increased i...,https://www.ons.gov.uk/releases/uktradefebruar...,"Exports of goods to EU countries, excluding pr...",0.742169,0.990666


In [189]:
for i in df_mps.index:
    df_mps['len_ratio'][i] = len(df_mps['bulletin_mps'][i]) / len(df_mps['roundup'][i])

In [155]:
df_mps=df_mps.sort_values(by=['rougef1'])

In [156]:
df_mps_high = df_mps[df_mps['rougef1'] > 0.15]
df_mps_low = df_mps[df_mps['rougef1'] < 0.15]

In [160]:
def p(df, n):
    print("MAIN POINTS: ", df.iloc[n]['bulletin_mps'])
    print("*********"*3)
    print("ROUNDUP: ", df.iloc[n]['roundup'])
    print("*********"*3)
    print("F1: ", df.iloc[n]['rougef1'])

p(df_mps_low, -1)

MAIN POINTS:  Of businesses not permanently stopped trading, 13% reported that vacancies were more difficult to fill in the last month compared with normal expectations for this time of year in late August 2021, up from 9% in early August 2021; 30% of businesses in the accommodation and food service activities industry reported vacancies being more difficult to fill than normal.
Half of currently trading businesses in the construction industry reported the prices of materials, goods or services bought in the last two weeks had increased more than normal price fluctuations, compared with approximately one-quarter of businesses across all industries.
The percentage of businesses currently trading in late August 2021 was 90%, a figure that has remained stable since late June 2021; the transportation and storage industry has remained the industry with the lowest percentage of businesses currently trading, at 78%.
The proportion of businesses' workforce reported to be on full or partial fur

In [168]:
df_mps_high_rat = df_mps_high[df_mps_high['len_ratio'] > 1]

In [206]:
df_mps_high_rat.shape

(53, 6)

In [177]:
df_mps_high_rat.to_csv('econ_high_rouge_len.csv', index=False)